In [1]:
import json
from functools import reduce
from collections import Counter
from collections import defaultdict
import numpy as np

In [10]:
def read_from_json(json_path):
    img_id2cat_id = defaultdict(list)
    
    json_data = json.load(open(json_path))
    image_ids = [im['id'] for im in json_data['images']]    
    catid2name = {category['id']: category['name'] for category in json_data['categories']}
    idx2catid = [catid for catid in catid2name]
    catid2idx = {catid: idx for idx, catid in enumerate(idx2catid)}
    
    for anno in json_data['annotations']:
        if anno['category_id'] not in img_id2cat_id[anno['image_id']]:
            img_id2cat_id[int(anno['image_id'])].append(anno['category_id'])
    # print('number of images with annotations', len(img_id2cat_id))
    return img_id2cat_id, catid2name, catid2idx, idx2catid

Only test on validation now. Training file too big need to use cluster

In [11]:
img_id2cat_id, catid2name, catid2idx, idx2catid = read_from_json('../mscoco/annotations/instances_val2017.json')

In [28]:
train_img_id2cat_id, _, _, _ = read_from_json('../mscoco/annotations/instances_train2017.json')

In [26]:
print(catid2name[idx2catid[0]])

person


In [27]:
# i = 191013
# i = 191288
# i = 191471
# i = 191580
# i = 191614
# i = 191672
# i = 191761
# i = 191845
###
for single_cat_id in img_id2cat_id[i]:
    print(catid2name[single_cat_id])
    print(catid2idx[single_cat_id])

In [37]:
for i in [191731,191126,191873,191296,191639,191376]:
    for single_cat_id in train_img_id2cat_id[i]:
        print('*'*30)
        print(i)
        print(catid2name[single_cat_id])
        print('cat id',single_cat_id)
        print(catid2idx[single_cat_id])

******************************
191731
banana
cat id 52
46
******************************
191126
bird
cat id 16
14
******************************
191873
truck
cat id 8
7
******************************
191873
traffic light
cat id 10
9
******************************
191873
person
cat id 1
0
******************************
191873
car
cat id 3
2
******************************
191296
fire hydrant
cat id 11
10
******************************
191639
cell phone
cat id 77
67
******************************
191639
person
cat id 1
0
******************************
191376
car
cat id 3
2
******************************
191376
bus
cat id 6
5
******************************
191376
person
cat id 1
0
******************************
191376
traffic light
cat id 10
9


In [ ]:
# use caption, time permitted
# cooc_from_json('../mscoco/annotations/captions_val2017.json')

In [44]:
import os
# supress tensorflow logging other than errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf


print(tf.__version__)
# 1.1.0

x = tf.placeholder(tf.int32, [5])
y = tf.placeholder(tf.int32, [5])
acc, acc_op = tf.metrics.accuracy(labels=x, predictions=y)

sess = tf.InteractiveSession()






1.12.0


In [57]:
# sess.run(tf.global_variables_initializer())
# sess.run(tf.local_variables_initializer())
v = sess.run([acc, acc_op], feed_dict={x: [1, 0, 0, 0, 0],
                                       y: [1, 0, 0, 0, 1]})
print(v)
# [0.0, 0.8]

v = sess.run(acc)
print(v)
# 0.8

[0.0, 0.26666668]
0.26666668


In [55]:

sess.run(tf.local_variables_initializer())
v = sess.run([acc, acc_op], feed_dict={x: [1, 0, 0, 0, 0],
                                       y: [0, 1, 1, 1, 1]})
print(v)
# [0.8, 0.4]

v = sess.run(acc)
print(v)

[0.0, 0.0]
0.0


In [56]:
v = sess.run([acc, acc_op], feed_dict={x: [1, 0, 0, 0, 0],
                                       y: [0, 1, 1, 1, 1]})
print(v)
# [0.8, 0.4]

v = sess.run(acc)
print(v)

[0.0, 0.0]
0.0


In [84]:
import tensorflow as tf
import numpy as np

y_true = np.array([[2, 1], [1, 2], [0, 1], [3, 1], [0, 1]]).astype(np.int64)
# y_true = np.array([[0,0,1,0], [0,1,0,0], [1,0,0,0], [0,0,0,1], [1,0,0,0]]).astype(np.int64)
y_true = tf.identity(y_true)

y_pred = np.array([[0.1, 0.2, 0.6, 0.1],
                   [0.8, 0.05, 0.1, 0.05],
                   [0.3, 0.4, 0.1, 0.2],
                   [0.6, 0.25, 0.1, 0.05],
                   [0.1, 0.2, 0.6, 0.1]
                   ]).astype(np.float32)
y_pred = tf.identity(y_pred)

_, m_ap = tf.metrics.sparse_average_precision_at_k(y_true, y_pred, 1)



In [127]:
a = tf.constant([[1, 10, 100],[2, 20, 200],[3, 30, 300]]) 
b = tf.constant([[4, 40, 400],[5, 50, 500],[6, 60, 600],[7, 70,700]]) 

tile_a = tf.tile(tf.expand_dims(a, 1), [1, tf.shape(b)[0], 1])  
tile_a = tf.reshape(tile_a, [-1, tf.shape(a)[1]])

tile_b = tf.tile(tf.expand_dims(b, 0), [tf.shape(a)[0], 1, 1]) 
tile_b = tf.reshape(tile_b, [-1, tf.shape(b)[1]])

# cartesian_product = tf.concat([tile_a, tile_b], axis=2) 
# cartesian_product = tf.reshape(cartesian_product, [-1, tf.shape(cartesian_product)[2], tf.shape(a)[1])
# cartesian_product = tf.transpose(cartesian_product, [1, 0, 2])

tile_a = tf.reshape(tile_a, [-1])
tile_a = tf.reshape(tile_a, [-1, 3])

cart = tf.Session().run(tile_a) 
tileb = tf.Session().run(tile_b)
# cart = tf.Session().run(tile_a)
print(tileb.shape)
print(tileb)


print(cart.shape) 
print(cart) 

(12, 3)
[[  4  40 400]
 [  5  50 500]
 [  6  60 600]
 [  7  70 700]
 [  4  40 400]
 [  5  50 500]
 [  6  60 600]
 [  7  70 700]
 [  4  40 400]
 [  5  50 500]
 [  6  60 600]
 [  7  70 700]]
(12, 3)
[[  1  10 100]
 [  1  10 100]
 [  1  10 100]
 [  1  10 100]
 [  2  20 200]
 [  2  20 200]
 [  2  20 200]
 [  2  20 200]
 [  3  30 300]
 [  3  30 300]
 [  3  30 300]
 [  3  30 300]]


### sess = tf.Session()
sess.run(tf.local_variables_initializer())

stream_vars = [i for i in tf.local_variables()]

tf_map = sess.run(m_ap)
print(tf_map)

# print((sess.run(stream_vars)))

# tmp_rank = tf.nn.top_k(y_pred,3)

# print(sess.run(tmp_rank)) 